In [7]:
%pip install beautifulsoup4 tqdm requests

  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/7c/e4/56027c4a6b4ae70ca9de302488c5ca95ad4a39e190093d6c1a8ace08341b/requests-2.32.4-py3-none-any.whl.metadata
  Obtaining dependency information for charset_normalizer<4,>=2 from https://files.pythonhosted.org/packages/39/f5/3b3836ca6064d0992c58c7561c6b6eee1b3892e9665d650c803bd5614522/charset_normalizer-3.4.3-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for idna<4,>=2.5 from https://files.pythonhosted.org/packages/76/c6/c88e154df9c4e1a2a66ccf0005a88dfb2650c1dffb6f5ce603dfbd452ce3/idna-3.10-py3-none-any.whl.metadata
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/a7/c2/fe1e52489ae3122415c51f387e221dd0773709bad6c6cdaa599e8a2c5185/urllib3-2.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/e5/48/1549795ba7742c948d2ad169c1c8cdbae65bc450d6c

In [8]:
# Importing necessary libraries
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests
import random

In [15]:
def get_word_list():
    #getting list of words from the url
    url='http://www.scrabble.org.au/words/fours.htm'
    words = BeautifulSoup(requests.get(url).text, 'html.parser').get_text().split("\n")
    # filter 4 letter words with unique characters and Return list of words
    return [word.strip() for word in words if len(word.strip()) == 4 and len(set(word.strip())) == 4 and word.strip().isupper() == True]

word_list = get_word_list()
print(f'Length: {len(word_list)}')


Length: 4309


In [16]:
# Feedback function to process the numbr of cats and dogs
def feedback(word_secret, word_guess):
    """
    This function compares the guessed word with the secret word and returns the number of correct letters in the correct position
    and the number of correct letters in the wrong position.
    """
    cat = dog = 0
    for i, j in zip(word_secret, word_guess):
        if i == j:
            dog += 1
        elif j in word_secret:
            cat += 1
    return cat, dog

In [17]:
class BanditSolver:
    def __init__(self, word_list, epsilon=0.2):
        self.word_list = word_list[:]  # candidate words
        self.epsilon = epsilon
        self.counts = {word: 0 for word in word_list}   # number of times each word was played
        self.values = {word: 0.0 for word in word_list} # estimated reward of each word

    def select_word(self):
        """ε-greedy selection: explore or exploit."""
        if random.random() < self.epsilon:
            return random.choice(self.word_list)
        return max(self.word_list, key=lambda word: self.values[word])

    def update(self, word_guess, reward):
        """Update bandit estimates after feedback."""
        self.counts[word_guess] += 1
        n = self.counts[word_guess]
        old_value = self.values[word_guess]
        self.values[word_guess] = old_value + (reward - old_value) / n

    def filter_word_list(self, word_guess, cats, dogs):
        """Eliminate inconsistent words from candidate space."""
        self.word_list = [
            word for word in self.word_list
            if feedback(word, word_guess) == (cats, dogs)
        ]


In [18]:
def play_game(word_secret, word_list, epsilon=0.2, max_rounds=20):
    solver = BanditSolver(word_list, epsilon)
    history = []

    for round_num in range(1, max_rounds+1):
        word_guess = solver.select_word()
        cats, dogs = feedback(word_secret, word_guess)
        reward = dogs + 0.5 * cats  # reward function

        solver.update(word_guess, reward)
        solver.filter_word_list(word_guess, cats, dogs)

        history.append((round_num, word_guess, cats, dogs))

        if dogs == 4:
            print(f"✅ Found secret word {word_secret} in {round_num} rounds!")
            return history

    print("❌ Failed to guess within limit.")
    return history


In [22]:
word_secret = input("Enter the secret word (4-letter, unique characters): ").upper().strip()
if word_secret not in word_list:
    print("The word is not in the list of valid words.")
    print(f"Secret (hidden for now): {word_secret}")

else:
    print(f"Secret word is: {word_secret}")
    game_history = play_game(word_secret, word_list)

    # Show game history
    for rnd, guess, cats, dogs in game_history:
        print(f"Round {rnd}: Guess = {guess}, Cats = {cats}, Dogs = {dogs}")


Secret word is: POSY
✅ Found secret word POSY in 7 rounds!
Round 1: Guess = ABED, Cats = 0, Dogs = 0
Round 2: Guess = CHIK, Cats = 0, Dogs = 0
Round 3: Guess = FLOG, Cats = 1, Dogs = 0
Round 4: Guess = GJUS, Cats = 1, Dogs = 0
Round 5: Guess = LUMP, Cats = 1, Dogs = 0
Round 6: Guess = MOST, Cats = 0, Dogs = 2
Round 7: Guess = POSY, Cats = 0, Dogs = 4


In [23]:
results = []
for secret in tqdm(word_list):
    history = play_game(secret, word_list)
    results.append((secret, len(history)))

# Show summary: number of rounds needed for each word
for secret, rounds in results:
    print(f"Secret: {secret}, Rounds: {rounds}")

  0%|          | 17/4309 [00:00<00:50, 85.82it/s]

✅ Found secret word ABED in 1 rounds!
✅ Found secret word ABET in 2 rounds!
✅ Found secret word ABID in 3 rounds!
✅ Found secret word ABLE in 2 rounds!
✅ Found secret word ABLY in 2 rounds!
✅ Found secret word ABOS in 3 rounds!
✅ Found secret word ABRI in 4 rounds!
✅ Found secret word ABUT in 5 rounds!
✅ Found secret word ABYE in 3 rounds!
✅ Found secret word ABYS in 3 rounds!
✅ Found secret word ACED in 3 rounds!
✅ Found secret word ACER in 5 rounds!
✅ Found secret word ACES in 4 rounds!
✅ Found secret word ACHE in 2 rounds!
✅ Found secret word ACHY in 2 rounds!
✅ Found secret word ACID in 5 rounds!
✅ Found secret word ACME in 3 rounds!
✅ Found secret word ACNE in 4 rounds!
✅ Found secret word ACRE in 7 rounds!
✅ Found secret word ACTS in 3 rounds!
✅ Found secret word ACYL in 4 rounds!
✅ Found secret word ADIT in 3 rounds!
✅ Found secret word ADOS in 5 rounds!


  1%|          | 41/4309 [00:00<00:41, 101.65it/s]

✅ Found secret word ADRY in 5 rounds!
✅ Found secret word ADZE in 2 rounds!
✅ Found secret word AEON in 3 rounds!
✅ Found secret word AERO in 4 rounds!
✅ Found secret word AERY in 6 rounds!
✅ Found secret word AESC in 3 rounds!
✅ Found secret word AFRO in 3 rounds!
✅ Found secret word AGED in 4 rounds!
✅ Found secret word AGEN in 5 rounds!
✅ Found secret word AGER in 4 rounds!
✅ Found secret word AGES in 5 rounds!
✅ Found secret word AGIN in 4 rounds!
✅ Found secret word AGIO in 4 rounds!
✅ Found secret word AGLU in 4 rounds!
✅ Found secret word AGLY in 6 rounds!
✅ Found secret word AGON in 5 rounds!
✅ Found secret word AGUE in 3 rounds!
✅ Found secret word AHED in 3 rounds!
✅ Found secret word AHEM in 5 rounds!
✅ Found secret word AHIS in 3 rounds!
✅ Found secret word AHOY in 3 rounds!
✅ Found secret word AIDE in 3 rounds!
✅ Found secret word AIDS in 5 rounds!
✅ Found secret word AILS in 3 rounds!
✅ Found secret word AIMS in 4 rounds!
✅ Found secret word AINE in 4 rounds!
✅ Found secr

  1%|▏         | 63/4309 [00:00<00:41, 103.28it/s]

...

✅ Found secret word YURT in 6 rounds!
✅ Found secret word YUTZ in 5 rounds!
✅ Found secret word YWIS in 7 rounds!
✅ Found secret word ZACK in 13 rounds!
✅ Found secret word ZAGS in 11 rounds!
✅ Found secret word ZANY in 8 rounds!
✅ Found secret word ZAPS in 11 rounds!
✅ Found secret word ZARF in 8 rounds!
✅ Found secret word ZARI in 9 rounds!
✅ Found secret word ZATI in 8 rounds!
✅ Found secret word ZEAL in 5 rounds!
✅ Found secret word ZEAS in 10 rounds!
✅ Found secret word ZEBU in 7 rounds!
✅ Found secret word ZEDS in 12 rounds!
✅ Found secret word ZEIN in 9 rounds!
✅ Found secret word ZEKS in 8 rounds!
✅ Found secret word ZELS in 11 rounds!
✅ Found secret word ZEPS in 9 rounds!
✅ Found secret word ZERK in 9 rounds!
✅ Found secret word ZERO in 9 rounds!
✅ Found secret word ZEST in 15 rounds!
✅ Found secret word ZETA in 9 rounds!


100%|█████████▉| 4294/4309 [00:51<00:00, 82.87it/s]

✅ Found secret word ZHOS in 8 rounds!
✅ Found secret word ZIGS in 11 rounds!
✅ Found secret word ZILA in 9 rounds!
✅ Found secret word ZIMB in 9 rounds!
✅ Found secret word ZINC in 5 rounds!
✅ Found secret word ZINE in 11 rounds!
✅ Found secret word ZING in 10 rounds!
✅ Found secret word ZINS in 14 rounds!
✅ Found secret word ZIPS in 8 rounds!
✅ Found secret word ZITE in 9 rounds!
✅ Found secret word ZITS in 10 rounds!
✅ Found secret word ZOBU in 6 rounds!
✅ Found secret word ZOEA in 6 rounds!
✅ Found secret word ZOIC in 4 rounds!
✅ Found secret word ZOLS in 9 rounds!
✅ Found secret word ZONA in 8 rounds!
✅ Found secret word ZONE in 8 rounds!
✅ Found secret word ZONK in 9 rounds!


100%|██████████| 4309/4309 [00:51<00:00, 83.07it/s]


✅ Found secret word ZORI in 6 rounds!
✅ Found secret word ZOUK in 10 rounds!
✅ Found secret word ZUPA in 6 rounds!
✅ Found secret word ZURF in 7 rounds!
✅ Found secret word ZYGA in 7 rounds!
✅ Found secret word ZYME in 7 rounds!
Secret: ABED, Rounds: 1
Secret: ABET, Rounds: 2
Secret: ABID, Rounds: 3
Secret: ABLE, Rounds: 2
Secret: ABLY, Rounds: 2
Secret: ABOS, Rounds: 3
Secret: ABRI, Rounds: 4
Secret: ABUT, Rounds: 5
Secret: ABYE, Rounds: 3
Secret: ABYS, Rounds: 3
Secret: ACED, Rounds: 3
Secret: ACER, Rounds: 5
Secret: ACES, Rounds: 4
Secret: ACHE, Rounds: 2
Secret: ACHY, Rounds: 2
Secret: ACID, Rounds: 5
Secret: ACME, Rounds: 3
Secret: ACNE, Rounds: 4
Secret: ACRE, Rounds: 7
Secret: ACTS, Rounds: 3
Secret: ACYL, Rounds: 4
Secret: ADIT, Rounds: 3
Secret: ADOS, Rounds: 5
Secret: ADRY, Rounds: 5
Secret: ADZE, Rounds: 2
Secret: AEON, Rounds: 3
Secret: AERO, Rounds: 4
Secret: AERY, Rounds: 6
Secret: AESC, Rounds: 3
Secret: AFRO, Rounds: 3
Secret: AGED, Rounds: 4
Secret: AGEN, Rounds: 5
Sec

In [24]:
# Calculate average number of guesses
total_guesses = sum(rounds for _, rounds in results)
average_guesses = total_guesses / len(results)
print(f"Average number of guesses: {average_guesses:.2f}")

# Find the toughest word (maximum guesses)
toughest_word, max_guesses = max(results, key=lambda x: x[1])
print(f"Toughest word: {toughest_word}, Number of guesses: {max_guesses}")

Average number of guesses: 6.64
Toughest word: VANE, Number of guesses: 15
